In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *

In [4]:
torch.cuda.set_device(0)

In [5]:
PATH = "/home/paperspace/data/dog-breed-identification/"
sz=224
arch=resnext101_64
batch_size=58

In [6]:
!ls {PATH}

labels.csv  sample_submission.csv  test  train


In [7]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)  # random 20% of the rows for training set
label_df = pd.read_csv(label_csv)

In [8]:
label_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [12]:
def get_data(sz, bs):
    tfms = tfms_from_model(
        arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(
        PATH,
        'train',
        label_csv,
        test_name='test',
        val_idxs=val_idxs,
        suffix='.jpg',
        tfms=tfms,
        bs=batch_size
    )
    return data if sz > 300 else data.resize(340, 'tmp')

In [15]:
data = get_data(sz, batch_size)

In [20]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 179/179 [02:07<00:00,  1.40it/s]


In [23]:
learn.fit(lrs=1e-2, n_cycle=5)

[0.      0.96672 0.37756 0.91906]                            
[1.      0.44078 0.37733 0.9114 ]                            
[2.      0.3072  0.29621 0.91236]                            
[3.      0.23339 0.2851  0.91763]                            
[4.      0.20644 0.3097  0.92146]                            



## Disabling Precomputing 

This still trains the last two layers, but activations are now recalculated for repeated images. This means that data augmentation now works!

In [24]:
learn.precompute=False

In [27]:
learn.fit(lrs=1e-2, n_cycle=5, cycle_len=1)

[0.      0.24176 0.23684 0.92768]                            
[1.      0.24799 0.23467 0.92816]                            
[2.      0.23083 0.23474 0.92624]                            
[3.      0.21546 0.23291 0.93056]                            
[4.      0.20346 0.22676 0.92577]                            



In [28]:
learn.save('224_pre')
learn.load('224_pre')

## Increasing the Size

Here we are getting images of a larger size, which will help generalize the model on top of providing more data

In [31]:
learn.set_data(get_data(299, batch_size))
learn.freeze()  # freeze the last layer

## Fiddling with Epochs

In order to more effectively utilize stochastic descent with restarts, we set `cycle_mult=2`. The first cycle is 1 epoch long, the second is 2 epochs long, and then third is 4 epochs long. Doing so prevents the network from "getting suck" in local minima

In [33]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)


  3%|▎         | 4/141 [00:06<03:41,  1.62s/it, loss=0.32]

 45%|████▍     | 63/141 [01:24<01:44,  1.34s/it, loss=0.208]


 90%|█████████ | 127/141 [02:45<00:18,  1.31s/it, loss=0.206]


100%|██████████| 141/141 [03:03<00:00,  1.30s/it, loss=0.212]
                                                             [0.      0.21239 0.2217  0.92864]

 35%|███▍      | 49/141 [01:05<02:02,  1.33s/it, loss=0.207]


 79%|███████▉  | 112/141 [02:27<00:38,  1.32s/it, loss=0.209]


100%|██████████| 141/141 [03:04<00:00,  1.31s/it, loss=0.204]
                                                             [1.      0.20351 0.22484 0.92768]

 24%|██▍       | 34/141 [00:45<02:24,  1.35s/it, loss=0.181]


...

100%|██████████| 141/141 [03:04<00:00,  1.31s/it, loss=0.162]
                                                             [4.      0.16214 0.22011 0.93151]

 38%|███▊      | 54/141 [01:11<01:54,  1.32s/it, loss=0.145]


 83%|████████▎ | 117/141 [02:34<00:31,  1.32s/it, loss=0.142]


100%|██████████| 141/141 [03:03<00:00,  1.30s/it, loss=0.148]
                                                             [5.      0.14778 0.22044 0.93343]

 28%|██▊       | 39/141 [00:51<02:15,  1.33s/it, loss=0.135]


 73%|███████▎  | 103/141 [02:14<00:49,  1.31s/it, loss=0.13]


100%|██████████| 141/141 [03:04<00:00,  1.31s/it, loss=0.138]
                                                             [6.      0.13846 0.21912 0.93534]



In [34]:
learn.save('299_pre')

In [114]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds, y),  metrics.log_loss(y, probs)


100%|██████████| 4/4 [03:02<00:00, 45.73s/it]
                                             

(0.9329745596868885, 0.20135554617167356)

In [115]:
y

array([ 19,  15,   7, ..., 109,   3,  28])

## Creating a Kaggle-Compatible Submission

We've achieved ~93.5% accuracy on the validation set. Probably much better than I would do, and I own a dog.

In [116]:
log_preds, y = learn.TTA(is_test=True)


100%|██████████| 4/4 [15:12<00:00, 228.20s/it]
                                              

In [128]:
probs = np.exp(log_preds)
ds = pd.DataFrame(probs)
ds.columns = data.classes
ds.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])
ds.head()


,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,de084b830010b6107215fef5d4a75b94,1.029411e-06,2.129471e-06,1.023886e-06,4.088997e-07,2.087486e-07,1.739787e-07,8.342177e-08,8.611963e-08,2.193610e-07,...,1.738356e-08,1.568734e-07,2.436195e-07,1.675257e-07,1.341819e-06,1.002695e-07,9.508463e-07,1.721296e-06,1.890647e-07,1.187914e-07
1,6b423ca7020e70eb05732843c5d2bad1,3.860963e-06,3.029199e-06,2.078257e-07,2.790143e-07,3.657066e-08,5.081838e-08,5.828688e-07,3.706798e-07,2.982284e-08,...,4.748391e-08,9.328693e-08,4.005446e-08,9.697933e-09,7.625179e-08,1.815584e-07,7.584066e-06,2.698672e-07,1.426708e-07,7.439682e-08
2,74aa7e201e0e93e13e87b986a7d31839,1.394899e-06,3.252299e-06,4.408828e-06,7.021684e-04,1.817512e-05,3.411037e-06,5.727566e-06,1.536449e-05,2.798669e-06,...,9.483220e-06,5.696977e-06,2.576118e-05,8.430353e-06,5.547687e-07,1.801967e-05,3.890948e-06,2.343402e-05,1.258153e-01,1.503929e-06
3,a079f72193264bc5685e5d28d7372680,1.763035e-04,2.028101e-06,6.044997e-05,7.010850e-06,2.165478e-06,8.832267e-07,2.510434e-06,9.676371e-07,1.224346e-06,...,6.294653e-06,6.903553e-06,3.077797e-06,1.459081e-06,4.962608e-06,2.524768e-06,2.148710e-04,7.612431e-07,1.225102e-03,2.939565e-05
4,583f7580fa5fec1266331fcf83b76fd6,1.221558e-07,4.687700e-09,1.070758e-08,1.847097e-07,6.405457e-09,6.285682e-07,1.405451e-09,1.399827e-09,1.464009e-10,...,3.531623e-10,3.570890e-09,2.155745e-09,9.217329e-10,7.855861e-08,1.021177e-08,2.492170e-09,4.251472e-08,6.953111e-07,6.762466e-09


In [152]:
SUBM = 'sub/'
os.makedirs(SUBM, exist_ok=True)
ds.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [153]:
FileLink(f'{SUBM}subm.gz')

/home/paperspace/kaggle/dog-breed-id/sub/subm.gz